### Финальный проект. Step 1.

In [122]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score

#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score
#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [123]:
df = pd.read_csv('train.csv')
df.head(3)

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   10000 non-null  int64  
 1   age                  10000 non-null  float64
 2   years_of_experience  10000 non-null  float64
 3   lesson_price         10000 non-null  float64
 4   qualification        10000 non-null  float64
 5   physics              10000 non-null  float64
 6   chemistry            10000 non-null  float64
 7   biology              10000 non-null  float64
 8   english              10000 non-null  float64
 9   geography            10000 non-null  float64
 10  history              10000 non-null  float64
 11  mean_exam_points     10000 non-null  float64
 12  choose               10000 non-null  int64  
dtypes: float64(11), int64(2)
memory usage: 1015.8 KB


In [125]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,10000.0,4999.5000,2886.895680,0.0,2499.75,4999.5,7499.25,9999.0
age,10000.0,45.8009,8.030274,23.0,40.00,46.0,51.00,68.0
years_of_experience,10000.0,1.9748,1.766883,0.0,0.00,2.0,3.00,9.0
lesson_price,10000.0,1702.4400,523.789062,200.0,1300.00,1550.0,2150.00,3950.0
qualification,10000.0,1.7243,0.798845,1.0,1.00,2.0,2.00,4.0
physics,10000.0,0.3706,0.482990,0.0,0.00,0.0,1.00,1.0
chemistry,10000.0,0.1215,0.326724,0.0,0.00,0.0,0.00,1.0
biology,10000.0,0.1172,0.321675,0.0,0.00,0.0,0.00,1.0
english,10000.0,0.0591,0.235824,0.0,0.00,0.0,0.00,1.0
geography,10000.0,0.0277,0.164120,0.0,0.00,0.0,0.00,1.0


In [126]:
features = ['lesson_price', 'mean_exam_points']
target = 'choose'

In [127]:
for feature in features:
    df[feature + '_pow'] = 1/(np.power(df[feature], 2) + 1)

In [128]:
df.columns

Index(['Id', 'age', 'years_of_experience', 'lesson_price', 'qualification',
       'physics', 'chemistry', 'biology', 'english', 'geography', 'history',
       'mean_exam_points', 'choose', 'lesson_price_pow',
       'mean_exam_points_pow'],
      dtype='object')

In [129]:
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    df['choose'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [130]:
features_fin = ['age', 'years_of_experience', 'lesson_price', 'qualification', 'physics', 'chemistry', 'biology', \
                'mean_exam_points', 'lesson_price_pow', 'mean_exam_points_pow']

In [131]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

In [132]:
%%time

pipeline = Pipeline([('features_selector', FeatureSelector(column=features_fin)), 
                     ('clf', LogisticRegression(max_iter=1000))])

pipeline.fit(X_train, y_train)

Wall time: 693 ms


Pipeline(steps=[('features_selector',
                 FeatureSelector(column=['age', 'years_of_experience',
                                         'lesson_price', 'qualification',
                                         'physics', 'chemistry', 'biology',
                                         'mean_exam_points', 'lesson_price_pow',
                                         'mean_exam_points_pow'])),
                ('clf', LogisticRegression(max_iter=1000))])

In [133]:
pipeline.steps

[('features_selector',
  FeatureSelector(column=['age', 'years_of_experience', 'lesson_price',
                          'qualification', 'physics', 'chemistry', 'biology',
                          'mean_exam_points', 'lesson_price_pow',
                          'mean_exam_points_pow'])),
 ('clf', LogisticRegression(max_iter=1000))]

In [134]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)